# Data preprocessing

## Summary of data analysis and findings


### Data exploration and analysis
- 11 columns from raw dataset
- 87489 rows (transactions)
- No null values
- Mix of float (3) and object (8) dtypes
- Exists some outliers for parking_fee
    - min = 0 (how does this make sense?)
    - max = 983 (this is very high compared to 75th percentile that is 39)
    - mean = 31
    - std = 39 (quite high compared to mean => high data variance)
    - Could for example remove outliers and standardize the data for easier training
- Latidute and longitude values are not in the correct range
    - min lat = -90 (but we observe -180)
    - offsetting wont help since the range is above 180, meaning I do not trust its values to be correctly measured
- Longitude is however in the correct range [-180, 180]
- Latitude and longitude data by itself does not really give much information and would require some kind of embedding model inbetween, that maps the coordinate point (latitude, longitude) to some feature space that includes information about the location and parking behaviour
- not an exact 50/50 split between private and corporate transactions, but they are in the same order of magnitude so stratified sampling is not super necessary
- There are 4 different currencies used in the dataset, huge majority is in SEK. 2 options which are both viable
    - convert all to SEK equivalent
    - remove transactions in other currencies
- area_type have 7 different values, "OnStreet" and "SurfaceLot" are the most common, while "EVC" and "CameraParkArea" are the least common
    - Before removing "EVC" and "CameraParkArea", we should check if there are any patterns (correlation) in the data that we could use to predict the account_type from those samples
- area_type would require categorical encoding (or one-hot encoding), before being used as input to the model.
- parking_id is unique as it is a primary key and equal to number of rows in the dataset, meaning it is completely useless for the model
- There are a total of 300 registered private and corporate parking users (accounts)
- There are a total of 1652 unique car ids used for parking transactions. Since this is larger than the number of parking users, some users have multiple cars, which is indicate of a business account (business has a fleet of cars). Could be useful to create a feature of number of used/registered cars per user
- Each parkinguser_id has only one account_type. If a person has both a private and a corporate account, then they make transactions using different parkinguser_ids!
- The time span of the dataset is very long (7 years). So normalizing with this time span is not a good idea. Some users could have been active for a long time while some could have been active for a short time.
- Better to normalize with the time span of each parkinguser_id


### Data cleaning
- no duplicate rows
- remove outliers in parking_fee (99th percentile)
- removed 0 parking_fee rows
- converted time data into datetime

### Feature engineering
- currency conversion
- parking duration (in hours)
- parking weekday/weekend
- registered cars per user
- parking count per user
- parking activity per car (parkings per day equivalent)


## Data exploration and analysis

In [1]:
import df_utils
import functionals
import pandas as pd

In [2]:
# Load data into Pandas DataFrame
data_path = "assignment-sample-data.csv"
df = pd.read_csv(data_path)

In [3]:
from IPython.display import display
pd.set_option('display.width', 1000)

In [4]:
# Show first entries
df.head()
#print(df.tail())

,parking_id,area_type,parking_start_time,parking_end_time,parking_fee,currency,parkinguser_id,car_id,lat,lon,account_type
0,fake_c28a323810,SurfaceLot,2015-03-06 19:55:41,2015-03-06 20:07:00,8.50,SEK,fake_bf5d9b530e,fake_130ae2aeb1,59.246370,18.077019,corporate
1,fake_76c21cf355,SurfaceLot,2015-03-06 18:08:20,2015-03-06 19:46:00,15.67,SEK,fake_bf5d9b530e,fake_130ae2aeb1,59.231789,18.083995,corporate
2,fake_995ed971a6,OnStreet,2017-07-21 09:55:42,2017-07-21 14:23:50,67.00,SEK,fake_3ba346a0cd,fake_f7a9d564d9,59.350331,18.096649,corporate
3,fake_6b81ea4f35,SurfaceLot,2017-07-24 07:21:12,2017-07-24 07:34:31,4.34,SEK,fake_ea19a50003,fake_fae7e31b34,59.315826,18.098355,corporate
4,fake_424b61e0eb,SurfaceLot,2015-03-09 12:05:46,2015-03-09 13:57:54,50.50,SEK,fake_1cc1970582,fake_0755f3c71f,59.320919,18.047513,corporate


- 11 columns (10 features, 1 target)
- I suspect that the rows are sorted by account_type since first 50 are corporate and last 50 are private. This means we should shuffle before training

In [5]:
shuffle_rows = True
if shuffle_rows:
    df = df.sample(frac=1).reset_index(drop=True)

In [6]:
print(df.columns)

Index(['parking_id', 'area_type', 'parking_start_time', 'parking_end_time', 'parking_fee', 'currency', 'parkinguser_id', 'car_id', 'lat', 'lon', 'account_type'], dtype='object')


In [7]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   parking_id          87489 non-null  object 
 1   area_type           87489 non-null  object 
 2   parking_start_time  87489 non-null  object 
 3   parking_end_time    87489 non-null  object 
 4   parking_fee         87489 non-null  float64
 5   currency            87489 non-null  object 
 6   parkinguser_id      87489 non-null  object 
 7   car_id              87489 non-null  object 
 8   lat                 87489 non-null  float64
 9   lon                 87489 non-null  float64
 10  account_type        87489 non-null  object 
dtypes: float64(3), object(8)
memory usage: 7.3+ MB
None


- 11 columns from raw dataset
- 87489 rows (transactions)
- No null values
- Mix of float (3) and object (8) dtypes

In [8]:
print(df.describe())

        parking_fee           lat           lon
count  87489.000000  87489.000000  87489.000000
mean      31.786153     58.786174     17.035432
std       39.338515      2.925073      2.639340
min        0.000000   -180.006219   -180.006783
25%        9.000000     59.292461     17.137638
50%       18.750000     59.332389     18.000587
75%       39.100000     59.360741     18.066440
max      983.000000     67.871132     54.009971


- Exists some outliers for parking_fee
    - min = 0 (how does this make sense?)
    - max = 983 (this is very high compared to 75th percentile that is 39)
    - mean = 31
    - std = 39 (quite high compared to mean => high data variance)
    - Could for example remove outliers and standardize the data for easier training
- Latidute and longitude values are not in the correct range
    - min lat = -90 (but we observe -180)
    - offsetting wont help since the range is above 180, meaning I do not trust its values to be correctly measured
- Longitude is however in the correct range [-180, 180]
- Latitude and longitude data by itself does not really give much information and would require some kind of embedding model inbetween, that maps the coordinate point (latitude, longitude) to some feature space that includes information about the location and parking behaviour

In [9]:
print(df["account_type"].unique())
print(df["account_type"].value_counts())
print(5*"-")
print(df["currency"].unique())
print(df["currency"].value_counts())
print(5*"-")
print(df["area_type"].unique())
print(df["area_type"].value_counts())
print(5*"-")
print(f"Number of rows in df: {len(df)}")
print(f"Number of unique parking_ids: {df['parking_id'].nunique()}")
print(f"Number of unique parkinguser_ids: {df['parkinguser_id'].nunique()}")
print(f"Number of unique car_ids: {df['car_id'].nunique()}")

['corporate' 'private']
account_type
private      57239
corporate    30250
Name: count, dtype: int64
-----
['SEK' 'NOK' 'DKK' 'EUR']
currency
SEK    86918
NOK      465
DKK       87
EUR       19
Name: count, dtype: int64
-----
['OnStreet' 'SurfaceLot' 'Administrative' 'UndergroundGarage'
 'AboveGroundGarage' 'EVC' 'CameraParkArea']
area_type
OnStreet             52747
SurfaceLot           26749
Administrative        6269
UndergroundGarage     1248
AboveGroundGarage      459
EVC                     14
CameraParkArea           3
Name: count, dtype: int64
-----
Number of rows in df: 87489
Number of unique parking_ids: 87489
Number of unique parkinguser_ids: 300
Number of unique car_ids: 1652


- not an exact 50/50 split between private and corporate transactions, but they are in the same order of magnitude so stratified sampling is not super necessary
- There are 4 different currencies used in the dataset, huge majority is in SEK. 2 options which are both viable
    - convert all to SEK equivalent
    - remove transactions in other currencies
- area_type have 7 different values, "OnStreet" and "SurfaceLot" are the most common, while "EVC" and "CameraParkArea" are the least common
    - Before removing "EVC" and "CameraParkArea", we should check if there are any patterns (correlation) in the data that we could use to predict the account_type from those samples
- area_type would require categorical encoding (or one-hot encoding), before being used as input to the model.
- parking_id is unique as it is a primary key and equal to number of rows in the dataset, meaning it is completely useless for the model
- There are a total of 300 registered private and corporate parking users (accounts)
- There are a total of 1652 unique car ids used for parking transactions. Since this is larger than the number of parking users, some users have multiple cars, which is indicate of a business account (business has a fleet of cars). Could be useful to create a feature of number of used/registered cars per user

### Check if transactions with "parkinguser_id" exits with multiple account types (corporate and private)

In [10]:
parkinguser_ids = {} # parkinguser_id -> account_type

# Loop over all transactions
for index, row in df.iterrows(): 
    user_id = row['parkinguser_id']
    account_type = row['account_type']

    if user_id not in parkinguser_ids: # If user_id is not in the dictionary, add it
        parkinguser_ids[user_id] = account_type
    else: # If user_id is already in the dictionary, check if the account_type is the same

        if parkinguser_ids[user_id] != account_type:
            print(f"User {user_id} has multiple account types: {parkinguser_ids[user_id]} and {account_type}")

- Each parkinguser_id has only one account_type. If a person has both a private and a corporate account, then they make transactions using different parkinguser_ids!

## Check if private accounts use cars that corporate accounts have used

In [11]:
violations = functionals.corporate_car(df)

for index, row in violations.iterrows():
    car_id = row['car_id']
    parkinguser_id = row['parkinguser_id']
    matching_transactions = df[(df['car_id'] == car_id) & (df['parkinguser_id'] != parkinguser_id)]
    print(f"Transactions with car_id {car_id}:")
    print(matching_transactions[['parking_id', 'parkinguser_id', 'car_id', 'account_type']])

Transactions with car_id fake_3c010b1ca5:
            parking_id   parkinguser_id           car_id account_type
14457  fake_5aea0993d9  fake_c28849a708  fake_3c010b1ca5    corporate
40760  fake_29e57ca780  fake_c28849a708  fake_3c010b1ca5    corporate
50205  fake_0885e9aca0  fake_c28849a708  fake_3c010b1ca5    corporate
Transactions with car_id fake_3c010b1ca5:
            parking_id   parkinguser_id           car_id account_type
14457  fake_5aea0993d9  fake_c28849a708  fake_3c010b1ca5    corporate
40760  fake_29e57ca780  fake_c28849a708  fake_3c010b1ca5    corporate
50205  fake_0885e9aca0  fake_c28849a708  fake_3c010b1ca5    corporate


### Check private accounts that 

In [12]:
shared_cars = functionals.shared_car(df)
len(shared_cars)
shared_cars

,car_id,user_count,user_ids
384,fake_3c010b1ca5,2,"[fake_c28849a708, fake_c8f3930a70]"
1050,fake_a46999824b,2,"[fake_cef59702cd, fake_b34f71b684]"
1472,fake_e877be7731,2,"[fake_b34f71b684, fake_cef59702cd]"
1567,fake_f52eb84892,2,"[fake_cef59702cd, fake_b34f71b684]"


### Max datetime range

In [13]:
# Data time range
df['parking_start_time'] = pd.to_datetime(df["parking_start_time"])
df['parking_end_time'] = pd.to_datetime(df["parking_end_time"])

# Find the earliest start time and latest end time
earliest_start = df['parking_start_time'].min()
latest_end = df['parking_end_time'].max()

# Calculate the total time span
time_span = latest_end - earliest_start

print(f"Earliest transaction start: {earliest_start}")
print(f"Latest transaction end: {latest_end}")
print(f"Total time span: {time_span.days} days, {time_span.seconds // 3600} hours")

Earliest transaction start: 2013-02-08 10:40:58
Latest transaction end: 2020-09-30 14:34:35
Total time span: 2791 days, 3 hours


- The time span of the dataset is very long (7 years). So normalizing with this time span is not a good idea. Some users could have been active for a long time while some could have been active for a short time.
- Better to normalize with the time span of each parkinguser_id

## Data cleaning

In [14]:
## Remove duplicate rows

# Create a temporary DataFrame without the parking_id column and check for duplicates
temp_df = df.drop(columns=['parking_id'])
duplicate_rows = temp_df[temp_df.duplicated(keep=False)]
print(f"Number of duplicate rows (excluding parking_id): {len(duplicate_rows)}")
if not duplicate_rows.empty:
    print("\nDuplicate rows (excluding parking_id):")
    display(duplicate_rows.sort_values(by=temp_df.columns.tolist()))
else:
    print("No duplicate rows found when excluding parking_id")


print(5*"-")

## Remove outliers in some features

# Calculate the 99th percentile of parking_fee
percentile = 0.99
fee_cutoff_value = df['parking_fee'].quantile(percentile)
rows_before = len(df)
df = df[(df['parking_fee'] > 0) & (df['parking_fee'] <= fee_cutoff_value)]
rows_removed = rows_before - len(df)
rows_removed_pct = (rows_removed / rows_before) * 100
print(f"Number of rows before filtering: {rows_before}")
print(f"Number of rows after filtering: {len(df)}")
print(f"Number of rows removed: {rows_removed} ({rows_removed_pct:.2f}%)")

print(5*"-")

## Convert date columns to datetime
df['parking_start_time'] = pd.to_datetime(df['parking_start_time'])
df['parking_end_time'] = pd.to_datetime(df['parking_end_time'])

Number of duplicate rows (excluding parking_id): 0
No duplicate rows found when excluding parking_id
-----
Number of rows before filtering: 87489
Number of rows after filtering: 81428
Number of rows removed: 6061 (6.93%)
-----


- no duplicate rows
- remove outliers in parking_fee (99th percentile)
- removed 0 parking_fee rows
- converted time data into datetime

## Feature Engineering

### Currency conversion

In [15]:
df["parking_fee_sek"] = df[["currency", "parking_fee"]].apply(df_utils.convert_currency, axis=1)
print(df[df["currency"] == "EUR"].head())

            parking_id   area_type  parking_start_time    parking_end_time  parking_fee currency   parkinguser_id           car_id        lat        lon account_type  parking_fee_sek
1549   fake_336bec8abf    OnStreet 2018-06-05 15:13:50 2018-06-05 15:30:00         1.05      EUR  fake_7047370fa5  fake_1fbe5a7dbe  60.397872  25.669604      private          11.6655
19469  fake_adc2bd373e    OnStreet 2019-08-06 07:20:04 2019-08-06 14:26:31         5.47      EUR  fake_ade3f3f432  fake_e996210230  46.530397  12.146991      private          60.7717
23319  fake_6b57bf21c3    OnStreet 2017-07-26 06:55:40 2017-07-26 12:30:00        15.41      EUR  fake_bd28523135  fake_0f86039c1b  60.469835  22.011726      private         171.2051
24636  fake_3f124ae99a    OnStreet 2020-07-29 07:19:44 2020-07-29 13:27:18         4.98      EUR  fake_ade3f3f432  fake_644ff1469a  46.548255  12.134466      private          55.3278
28197  fake_56f5699a58  SurfaceLot 2019-01-05 16:37:22 2019-01-05 17:30:00         2.

### Parking duration (in hours)

In [16]:
df["parking_duration"] = df[["parking_start_time", "parking_end_time"]].apply(lambda x: (x["parking_end_time"] - x["parking_start_time"]).total_seconds() / 3600, axis=1)

### Weekday calculation (weekday = 1, weekend = 0)



In [17]:
df["weekday"] = df.apply(df_utils.get_weekday, axis=1)

### Unique cars used per account

In [18]:
# Count unique cars per user
user_car_counts = df.groupby("parkinguser_id")["car_id"].nunique()

# Map the counts back to the original dataframe
df["registered_cars"] = df["parkinguser_id"].map(user_car_counts)

# Display the first few rows to verify
print(df[["parking_id","parkinguser_id", "car_id", "registered_cars", "account_type"]].head(10))

        parking_id   parkinguser_id           car_id  registered_cars account_type
0  fake_273c852869  fake_c6a7eb1142  fake_bbe4909d5a                2    corporate
1  fake_11cbce8cc5  fake_6e03a1bfc8  fake_160f8ea2b1                3      private
2  fake_2a49931a5c  fake_f6adf105e7  fake_1172ecde57                1    corporate
3  fake_1f3782e52d  fake_d317e5c7a5  fake_8839d145c1                6    corporate
4  fake_c4c4884683  fake_bdbd8ad11c  fake_73f0637761                9    corporate
5  fake_5d518a2035  fake_5992b94be2  fake_162f77c7a6                3      private
6  fake_ff607e57e9  fake_a111101e08  fake_75b27675d8                6      private
7  fake_a48a8f9cf6  fake_9ad34bc6d8  fake_8db121fbff                2      private
8  fake_af16b3b212  fake_f8d0fde534  fake_2b93a70524                2      private
9  fake_12feb1064d  fake_0703d99383  fake_e1ad80583c                8      private


### Parking count

In [19]:
# Count total parking transactions per user
user_parking_counts = df['parkinguser_id'].value_counts()
df['n_parkings'] = df['parkinguser_id'].map(user_parking_counts)
print(df[['parkinguser_id', 'n_parkings', "account_type"]].head(10))

    parkinguser_id  n_parkings account_type
0  fake_c6a7eb1142         339    corporate
1  fake_6e03a1bfc8          80      private
2  fake_f6adf105e7         159    corporate
3  fake_d317e5c7a5        1965    corporate
4  fake_bdbd8ad11c        2061    corporate
5  fake_5992b94be2         118      private
6  fake_a111101e08         167      private
7  fake_9ad34bc6d8         210      private
8  fake_f8d0fde534         317      private
9  fake_0703d99383         381      private


### Parking activity (normalized w.r.t days)

In [20]:
# Normalize parking frequency with respect to days used


# Ensure the timestamp columns are in datetime format
df["parking_start_time"] = pd.to_datetime(df["parking_start_time"])
df["parking_end_time"] = pd.to_datetime(df["parking_end_time"])

# Calculate account age in days for each user
user_first_last = df.groupby("parkinguser_id").agg(
    first_parking=("parking_start_time", "min"),
    last_parking=("parking_end_time", "max")
)

# Calculate account age in days (add 1 to avoid division by zero for single transactions)
user_first_last["account_age_days"] = (user_first_last["last_parking"] - user_first_last["first_parking"]).dt.days + 1

# Get total parkings per user (we already have this in "n_parkings")
# Calculate parking activity (parkings per day)
user_first_last["parking_activity"] = df.groupby("parkinguser_id").size() / user_first_last["account_age_days"]

# Map the parking_activity back to the original dataframe
df = df.merge(
    user_first_last[["parking_activity"]],
    left_on="parkinguser_id",
    right_index=True,
    how="left"
)

# Display the results
print("Sample of user parking activity (parkings per day):")
print(df[["parkinguser_id", "n_parkings", "parking_activity", "account_type"]].head(10))
print("\nSummary statistics for parking_activity:")
print(df["parking_activity"].describe())

Sample of user parking activity (parkings per day):
    parkinguser_id  n_parkings  parking_activity account_type
0  fake_c6a7eb1142         339          0.304036    corporate
1  fake_6e03a1bfc8          80          0.037488      private
2  fake_f6adf105e7         159          0.249216    corporate
3  fake_d317e5c7a5        1965          1.234296    corporate
4  fake_bdbd8ad11c        2061          0.836105    corporate
5  fake_5992b94be2         118          0.148055      private
6  fake_a111101e08         167          0.665339      private
7  fake_9ad34bc6d8         210          0.188510      private
8  fake_f8d0fde534         317          0.236567      private
9  fake_0703d99383         381          0.196189      private

Summary statistics for parking_activity:
count    81428.000000
mean         0.471321
std          0.330562
min          0.005571
25%          0.195626
50%          0.370934
75%          0.742671
max          1.234296
Name: parking_activity, dtype: float64


- This feature has pros and cons. If the parking activity is uniform then it will measure average activity pretty well. However, if the activity is not uniform (such as a big break between parkings) then the acitivity will be underestimated.
- Parking activity could be a good way to measure activity per day is a better measure compared to total parking transactions since a private user that has used the app for a long time will have more parking transactions than a new business user that has just started using the app.

## Before

In [21]:
df.head()

,parking_id,area_type,parking_start_time,parking_end_time,parking_fee,currency,parkinguser_id,car_id,lat,lon,account_type,parking_fee_sek,parking_duration,weekday,registered_cars,n_parkings,parking_activity
0,fake_273c852869,OnStreet,2015-10-21 12:09:16,2015-10-21 14:56:47,49.50,SEK,fake_c6a7eb1142,fake_bbe4909d5a,59.327621,18.080146,corporate,49.50,2.791944,1,2,339,0.304036
1,fake_11cbce8cc5,OnStreet,2016-09-12 12:53:16,2016-09-12 14:56:00,61.30,SEK,fake_6e03a1bfc8,fake_160f8ea2b1,59.330787,18.084943,private,61.30,2.045556,1,3,80,0.037488
2,fake_2a49931a5c,OnStreet,2020-06-11 13:33:11,2020-06-12 08:40:45,20.58,SEK,fake_f6adf105e7,fake_1172ecde57,58.720198,14.500147,corporate,20.58,19.126111,1,1,159,0.249216
3,fake_1f3782e52d,SurfaceLot,2019-03-28 16:47:29,2019-03-28 17:02:06,2.00,SEK,fake_d317e5c7a5,fake_8839d145c1,59.327338,17.956198,corporate,2.00,0.243611,1,6,1965,1.234296
4,fake_c4c4884683,OnStreet,2018-03-17 01:33:27,2018-03-17 21:54:00,60.00,SEK,fake_bdbd8ad11c,fake_73f0637761,59.342401,18.093242,corporate,60.00,20.342500,0,9,2061,0.836105


## Final Visualization (After)

In [22]:
df = df.drop(columns=['parking_id', "parking_fee", 'lat', 'lon', "currency", "car_id", "parking_start_time", "parking_end_time"])
df = df[['parkinguser_id'] + [col for col in df.columns if col not in ['parkinguser_id', 'account_type']] + ['account_type']]
df.head()

,parkinguser_id,area_type,parking_fee_sek,parking_duration,weekday,registered_cars,n_parkings,parking_activity,account_type
0,fake_c6a7eb1142,OnStreet,49.50,2.791944,1,2,339,0.304036,corporate
1,fake_6e03a1bfc8,OnStreet,61.30,2.045556,1,3,80,0.037488,private
2,fake_f6adf105e7,OnStreet,20.58,19.126111,1,1,159,0.249216,corporate
3,fake_d317e5c7a5,SurfaceLot,2.00,0.243611,1,6,1965,1.234296,corporate
4,fake_bdbd8ad11c,OnStreet,60.00,20.342500,0,9,2061,0.836105,corporate


## Save processed data

In [23]:
save_df = True
# Save cleaned dataframe
if save_df:
    clean_path = data_path[:-4] + '-cleaned.csv'
    df.to_csv(clean_path, index=False)

## Future work 

- Using longitude and latitdue coordinates, can we be useful if we have some kind of embedding models that can translate this information to a richer representation. Right now the raw coordinates are not very useful to the model, especially standard ML models. 
- Including start time into the model input could be useful but one needs to be careful how to encode that. Using the absolute date is bad since it has high range of value. One should opt to use the periodicty in the 24 hour clock. For example add a feature that shows the offset to midnight. (2018-11-27 15:17:50 -> 15. hours). But this is an issue since 23:59 and 00:01 distance. This adding as trigonometric transformation such as sine or cosine with 24h period time would include this periodicity and limit the range of values. Since we have parking duration we thus do not need to include the end time since this is redundant information.